### Importing the Libraries

In [39]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [40]:
tf.__version__ 

'2.12.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [41]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [42]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [43]:
!wget https://github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/archive/refs/heads/main.zip



--2023-04-01 17:10:35--  https://github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/zip/refs/heads/main [following]
--2023-04-01 17:10:35--  https://codeload.github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip.2’

main.zip.2              [              <=>   ] 421.79M  26.3MB/s    in 16s     

2023-04-01 17:10:52 (25.7 MB/s) - ‘main.zip.2’ saved [442279055]



In [44]:
!unzip main.zip

Archive:  main.zip
5d90dd4f608245cabf41f540a12fd9b1a2f3a327
replace Sign-Language-To-Text-Conversion1-main/dataSet/testingData/0/108.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Creating the Training set

In [45]:
!mkdir trainingData

mkdir: cannot create directory ‘trainingData’: File exists


In [46]:
!mkdir testingData

mkdir: cannot create directory ‘testingData’: File exists


In [47]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/D' 'trainingData'

In [48]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/R' 'trainingData'

In [49]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/U' 'trainingData'

In [50]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/D' 'testingData'

In [51]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/R' 'testingData'

In [52]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/U' 'testingData'

In [94]:
training_set = train_datagen.flow_from_directory('trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 1401 images belonging to 3 classes.


In [95]:
test_set = test_datagen.flow_from_directory('testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 465 images belonging to 3 classes.


# Part 2: Building CNN


# Initializing CNN

In [96]:
classifier = tf.keras.models.Sequential()

#Step 1: Convolution

In [97]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#Step 2:  Pooling

In [98]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

# Adding a second convolution layer

In [99]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

# Step 3: Flattering

In [100]:
classifier.add(tf.keras.layers.Flatten())

# Step 4: Full Connection

In [107]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=32, activation='relu')) # softmax for more than 2
classifier.add(tf.keras.layers.Dense(units=16, activation='relu')) # softmax for more than 2
classifier.add(tf.keras.layers.Dense(units=3, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [108]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [109]:
classifier.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 32768)             0         
                                                                 
 dense_16 (Dense)            (None, 128)              

In [110]:
classifier

In [112]:
classifier.fit(training_set,
                  epochs = 30,
                  validation_data = test_set)

Epoch 1/30
141/141 [==============================] - 38s 269ms/step - loss: 0.0876 - accuracy: 0.9872 - val_loss: 0.0636 - val_accuracy: 0.9892
Epoch 2/30
141/141 [==============================] - 43s 307ms/step - loss: 0.1445 - accuracy: 0.9729 - val_loss: 0.1075 - val_accuracy: 0.9849
Epoch 3/30
141/141 [==============================] - 40s 280ms/step - loss: 0.1648 - accuracy: 0.9700 - val_loss: 0.0471 - val_accuracy: 0.9935
Epoch 4/30
141/141 [==============================] - 35s 250ms/step - loss: 0.1380 - accuracy: 0.9743 - val_loss: 0.0833 - val_accuracy: 0.9935
Epoch 5/30
141/141 [==============================] - 36s 256ms/step - loss: 0.1059 - accuracy: 0.9807 - val_loss: 0.0242 - val_accuracy: 0.9957
Epoch 6/30
141/141 [==============================] - 38s 271ms/step - loss: 0.0792 - accuracy: 0.9850 - val_loss: 0.0481 - val_accuracy: 0.9935
Epoch 7/30
141/141 [==============================] - 39s 277ms/step - loss: 0.1116 - accuracy: 0.9793 - val_loss: 0.0297 - val_ac

#### Saving the Model

In [113]:
model_json = classifier.to_json()
with open("model_bw-dru.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_bw-dru.h5')
print('Weights saved')

Model Saved
Weights saved


In [ ]:
classifier.fit(training_set,
                  epochs = 20,
                  validation_data = test_set)

In [ ]:
model2_json = classifier.to_json()
with open("model2_new.json", "w") as json_file:
    json_file.write(model2_json)
print('Model Saved')
classifier.save_weights('model2_new.h5')
print('Weights saved')

In [ ]:
!cp '/content/model2_new.h5' '/content/drive/MyDrive'